In [1]:
# https://github.com/deepmind/kinetics-i3d/files/3879458/convert_to_flow.ipynb.zip

import os
import numpy as np
import cv2
from glob import glob
from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

_IMAGE_SIZE = 256

def cal_for_frames(video_path): # frame/test_rgb/
    frames = glob(os.path.join(video_path, '*.jpg'))
    frames.sort()

    flow = []
    prev = cv2.imread(frames[0])
    prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    prev = cv2.resize(prev,(224,224))
    for i, frame_curr in enumerate(frames):
        if i%5==0:
            curr = cv2.imread(frame_curr)
            curr = cv2.resize(curr,(224,224))
            curr = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
            tmp_flow = compute_TVL1(prev, curr)
            flow.append(tmp_flow)
            prev = curr

    return flow

def compute_TVL1(prev, curr, bound=15):
    """Compute the TV-L1 optical flow."""
    TVL1 = cv2.optflow.DualTVL1OpticalFlow_create()
    flow = TVL1.calc(prev, curr, None)
    flow = np.clip(flow, -20,20) #default values are +20 and -20
    #print(flow)
    assert flow.dtype == np.float32

    flow = (flow + bound) * (255.0 / (2*bound))
    flow = np.round(flow).astype(int)
    flow[flow >= 255] = 255
    flow[flow <= 0] = 0

    return flow

# def save_flow(video_flows, flow_path): # flow, test/test_flow/
def save_flow(video_flows): # flow, test/test_flow/
    # The optical flows are generated in 3D. However for I3D only first two channels are used. u is the first channel
    # and v is the second channel. Both u and v are saved in separate folders in the flow_path directory.
    # The u and v folders will be generated by calling create_path() function
    n_py=[]
    for i, flow in enumerate(video_flows):
        # dont save
        # cv2.imwrite(os.path.join(flow_path.format('u'), "{:06d}.jpg".format(i)), flow[:, :, 0])
        # cv2.imwrite(os.path.join(flow_path.format('v'), "{:06d}.jpg".format(i)), flow[:, :, 1])
        n_py.append(np.dstack([flow[:, :, 0],flow[:, :, 1]]))
    return n_py

def extract_flow(args):
    video_path, flow_path = args
    flow = cal_for_frames(video_path)
    save_flow(flow, flow_path)
    print('complete:' + flow_path)
    return

def create_paths(base_path):
    # activity_path = os.path.join(base_path, activity) # train/activity
    # rgb_folder = os.makedirs('{}/{}_rgb'.format(activity_path, activity)) # train/activity/activity_rgb
    # flow_folder1 = os.makedirs('{}/{}_flow/u'.format(activity_path, activity)) # train/activity/activity_flow/u
    # flow_folder2 = os.makedirs('{}/{}_flow/v'.format(activity_path, activity)) # train/activity/activity_flow/v
    save_folder = os.path.join(base_path, "output")
    src_folder = os.path.join(base_path, "src")
    c_lists = glob(os.path.join(src_folder,"*"))
    for cls in c_lists:
        # v_lists = glob(os.path.join(cls,"*"))
        cls_name = cls.split("/")[-1]

        flow_dir = save_folder + "/flow/" + cls_name
        frame_dir = save_folder + "/frame/" + cls_name
        npyflow_dir = save_folder + "/npyflow/" + cls_name
        rgb_dir = save_folder + "/rgb/" + cls_name

        os.mkdir(flow_dir)
        os.mkdir(frame_dir)
        os.mkdir(npyflow_dir)
        os.mkdir(rgb_dir)

        v_lists = glob(os.path.join(cls, "*"))
        for v in v_lists:
            video = v.split("/")[-1][:-4]
            os.mkdir(os.path.join(flow_dir, video))
            os.mkdir(os.path.join(frame_dir, video))
            os.mkdir(os.path.join(npyflow_dir, video))
            os.mkdir(os.path.join(rgb_dir, video))

def vid_to_image(video_path, frame_path):
    print("video path:", video_path)
    print("frame path:", frame_path)

    cls_lists = glob(os.path.join(video_path, "*"))
    print("cls lists:", cls_lists)
    for cls in cls_lists:
        cls_name = cls.split("/")[-1]
        save_dir = os.path.join(frame_path, cls_name)

        v_lists = glob(os.path.join(cls, "*"))
        for v in v_lists:
            vid_name = v.split("/")[-1][:-4]
            vidcap = cv2.VideoCapture(v)
            success, image = vidcap.read()
            count = 0
            frame_no = 0
            print("save frame path:", os.path.join(save_dir, vid_name))
            while success:
                vidcap.set(1, frame_no)
                # print(os.path.join(frame_path,"frame%d.jpg"))
                cv2.imwrite(os.path.join(os.path.join(save_dir, vid_name), "frame%d.jpg") % count, image)  # save frame as JPEG file
                success, image = vidcap.read()
                # print('Read a new frame: ', success)
                count += 1
                frame_no += 1

    '''
    # print("base path/file/frame_path", base_path, file, frame_path)
    folder=file.split('.')[0]
    #print(folder)
    #os.mkdir(os.path.join(base_path,folder))
    vidcap = cv2.VideoCapture(file)
    success,image = vidcap.read()
    count = 0
    frame_no=0
    print("frame path:", frame_path)
    while success:
            vidcap.set(1,frame_no)
            #print(os.path.join(frame_path,"frame%d.jpg"))
            cv2.imwrite(os.path.join(frame_path,"frame%d.jpg") % count, image)     # save frame as JPEG file
            success,image = vidcap.read()
            #print('Read a new frame: ', success)
            count += 1
            frame_no+=1
    '''

In [3]:
## Create npy file for rgb files as described in deepmind I3D
def norm_rgb(rgb_path, nchannel):
    # print(video_path)
    npy_file = []
    frames = glob(os.path.join(rgb_path, '*.jpg'))
    frames.sort()
    # print(len(frames))
    # print("rgb_path:", rgb_path)
    for i, frame in enumerate(frames):
        if i%5==0:
            img = cv2.imread(frame)
            img_new = (cv2.resize(img, (224, 224))).astype(float)
            # img_norm = np.divide(2 * (img_new - img_new.min()), (img_new.max() - img_new.min())) - 1
            npy_file.append(img_new)
            # npy_file.append(img_norm)

    try:
        npy_file = np.reshape(np.asarray(npy_file), (1, len(frames)//5, 224, 224, nchannel))
    except:
        npy_file = np.reshape(np.asarray(npy_file), (1, len(frames)//5+1, 224, 224, nchannel))
    return npy_file

In [7]:
# 4:20
if __name__ == "__main__":
#     01. create class folder
    #base_path = "/home/veryyoung/문서/optical_flow_extractor/extract"
    base_path = "/home/hdd1/data/extract"
    src_path = os.path.join(base_path, "src")
    output_path = os.path.join(base_path, "output")

    cls_lists = glob(os.path.join(src_path, "*"))
    print("cls lists:", cls_lists, "\n")
    
    cls_lists.sort()

#     02. calculate optical flow

#     for i, cls in enumerate(cls_lists):
    cls_name = cls.split("/")[-1]
    print("class: {}/{}\t{}".format(i, len(cls_lists), cls_name))
    n_cls_path = os.path.join(output_path, cls_name)

    video_lists = glob(os.path.join(cls, "*"))

    v = "/home/hdd1/data/extract/src/bending/getty-woman-harvesting-cauliflower-on-organic-farm-video-id593422949_5"

#         for j, v in enumerate(video_lists):
    vid_name = v.split("/")[-1]
    des_path = os.path.join(n_cls_path, vid_name)

    # save at each dir
    np_file_rgb = norm_rgb(v, 3)
#             np.save(des_path + '/rgb.npy', np_file_rgb)

cls lists: ['/home/hdd1/data/extract/src/sleeping', '/home/hdd1/data/extract/src/laughing', '/home/hdd1/data/extract/src/smoking', '/home/hdd1/data/extract/src/drinking', '/home/hdd1/data/extract/src/tying', '/home/hdd1/data/extract/src/writing', '/home/hdd1/data/extract/src/winking', '/home/hdd1/data/extract/src/slapping', '/home/hdd1/data/extract/src/smiling', '/home/hdd1/data/extract/src/typing', '/home/hdd1/data/extract/src/driving', '/home/hdd1/data/extract/src/pointing', '/home/hdd1/data/extract/src/crying', '/home/hdd1/data/extract/src/running', '/home/hdd1/data/extract/src/calling', '/home/hdd1/data/extract/src/sitting', '/home/hdd1/data/extract/src/standing', '/home/hdd1/data/extract/src/squinting', '/home/hdd1/data/extract/src/bending', '/home/hdd1/data/extract/src/talking', '/home/hdd1/data/extract/src/walking', '/home/hdd1/data/extract/src/reading', '/home/hdd1/data/extract/src/folding', '/home/hdd1/data/extract/src/frowning'] 

class: 3/24	drinking


In [9]:
np_file_rgb.shape

(1, 19, 224, 224, 3)